In [1]:
#installed with pip
from pandas import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.error import HTTPError
from collections import namedtuple

# built-in libraries
import datetime
import re
import sqlite3
import urllib
import csv
import datetime

In [2]:
# Read state summary report for District into Pandas df; filter for needed fields and assign headers
insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                   usecols=[2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 17, 18, 80, 81], encoding="ISO-8859-1")
insp.columns = ["county", "licnum", "sitename", "streetaddy", "cityaddy", "zip",
                "inspnum", "insptype", "inspdispos", "inspdate", "totalvio", "highvio", "licid", "visitid"]
county_sought = 'Marion' # uncomment if particular county sought
insp = insp[(insp.county == county_sought)] # uncomment if particular county sought
insp.sitename = insp.sitename.str.title() #titlecase sitename
insp.streetaddy = insp.streetaddy.str.title() #titlecase street adress
insp.cityaddy = insp.cityaddy.str.title() #titlecase city
insp = insp.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) #strip whitespace, replace empty vals
insp['visitid'] = insp['visitid'].apply(int) # so it can be filtered against df below

# Read in records from database of earlier reports, create df to filter against new reports in df above.
conn = sqlite3.connect("rinspect.sqlite")
df = pd.read_sql_query("select * from fdinsp;", conn)
df
conn.close()

unique_vals = insp[~insp.visitid.isin(df.visitid)] #filter

In [3]:
# Build a list of URLs to the inspectors detailed reports that will get scraped
result = []
result_for_urls = result # for url list
result_for_list = result.append("NULL") # later into db, space for user-input datetime

# takes LicenseID and VisitID, passes it into the urls for detailed reports later
for index, rows in unique_vals.iterrows():
    visitid = rows['visitid']
    licid = rows['licid']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &id= %s" % (visitid, licid)
    urls = urls.replace(' ', '')
    result.append(urls)
urlList = result
urlList.pop(0) # get rid of first "Null" from append above
len(urlList)

265

In [4]:
# Place data from state summary report df into database
# first, interate through the df to return tuples
var = list(unique_vals.itertuples(index='visitid', name=None))
# populate database table for inspection summmary reports
sqlite_file = 'rinspect.sqlite'
fdinsp_table = 'fdinsp' # table for summary data
id_field = 'visitid' # ID column
time_now = 'now' # column for user-input timestamp; until then = NULL

# connect to database and insert new summary report data
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.executemany('''INSERT OR IGNORE INTO fdinsp (librow, county, licnum, sitename,
              streetaddy, cityaddy, zip, inspnum, insptype, inspdispos,
              inspdate, totalvio, highvio, licid, visitid)
              VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', var)
conn.commit()
conn.close()

In [ ]:
# This getting close for getting observations details from a single url
file_path = 'observations.txt'
visitid = inspurl.split("VisitID=")[1].split("&")[0]
html = urlopen(url)
soup = BeautifulSoup(html.read(), 'html.parser')
details = soup.findAll('font', {'face': 'verdana'})[42:]
# regex string matches for tags in the soup; not sure if to replace here or later
regex = re.compile('\<.*\>') 
match = regex.search(str(details))
new_details = details
new_details = regex.sub('\<.*\>', new_details)
# writes out file
# need to append with visitid, and other details from database, into narrative;
# put observations into the database? If so, need to separate them into different fields
with open('observations.txt'.format(file_path), mode='wt', encoding='utf-8') as file:
    file.write(str(new_details))

In [ ]:
# or take a different approach by building a dictionary of observations;
# but this isn't close to working
# function to scrape inspections from list of urls above, place in data structure
def make_details_dict(url):
    global detailsdict
    inspurl = urlList
    # extract VisitID number from url; will be pk in database
    visitid.append(inspurl.split("VisitID=")[1].split("&")[0]) 
    # or ...
    # visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
    html = urlopen(url)
    soup = bs(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
    detailsdict = {
        'visitid' : visitid,
        'observed': details,
    }
    result.append(detailsdict)    
    return details_dict

# iterate through urls, calling function above to make dictionary
for url in urlList:
    make_details_dict(url)
                          
# need to send this stuff to dict once it's working

In [6]:
# a different try for a dictionary; do I need to work with classes to build this?
# the exceptions seem to work ok, but will need "Nulls" not None into database if we do that.
details_dict = {}
def make_details_list(urlList):
    inspurl = urlList
    visitid = visitid.append(inspurl.split("VisitID=")[1].split("&")[0])
    html = urlopen(url)
    soup = bs(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
    result = []
        
    try:
        observed1 = details[32].text
    except HTTPError as e:
        return None
    return observed1
    try:
        observed2 = details[34].text
    except HTTPError as e:
        return None
    return observed2
    try:
        observed3 = details[36].text
    except HTTPError as e:
        return None
    return observed3
    try:
        observed4 = details[38].text
    except HTTPError as e:
        return None
    return observed4
    try:
        observed5 = details[40].text
    except HTTPError as e:
        return None
    return observed5
    try:
        observed6 = details[42].text
    except HTTPError as e:
        return None
    return observed6
    try:
        observed7 = details[44].text
    except HTTPError as e:
        return None
    return observed7
    try:
        observed8 = details[46].text
    except HTTPError as e:
        return None
    return observed8
        
    result.append(details_list)
    return details_list    

for url in urlList:
    make_details_list(urlList)
    



AttributeError: 'list' object has no attribute 'rfind'